## CrewAI and Together AI Agent

In [1]:
!pip install crewai crewai[tools] gspread oauth2client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.0/677.0 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 8.6 MB/s e

## Google Sheets

* ตัวเสียตังค์ - https://docs.crewai.com/en/enterprise/integrations/google_sheets

* API Models ของ Together : https://www.together.ai/pricing#inference

NOTE : Google Sheets ห้ามสร้าง Tool แยกสองตัว เพราะว่ามันไม่สามารถส่ง Dataframe ข้ามกันได้ ทางที่ดีสุดคือ สร้างเป็น tools เดียวกันไปเลย หรือไม่ก็เปิด MCP สักตัว

In [71]:
import os
from crewai import LLM, Task, Agent, Crew
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from datetime import datetime
import pytz
from typing import Type
import pandas as pd

# Google Sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient import discovery

# Load credentials and authorize gspread for google sheets
# Assuming 'client_secret.json' is in the same directory or accessible path
try:
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
    client = gspread.authorize(credentials)
except FileNotFoundError:
    print("Error: 'client_secret.json' not found. Please make sure the file is in the correct directory.")
    client = None # Set client to None if credentials file is not found

class TimeTool(BaseTool):
    name: str = "time_tool"
    description: str = "Useful for getting current date and time information in Thailand timezone. Input should be a natural language query about time or date."

    def _run(self, *args, **kwargs):
        query = kwargs.get("query")
        thailand_tz = pytz.timezone('Asia/Bangkok')
        current_time = datetime.now(thailand_tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"Current time in Thailand: {current_time}"

class GoogleSheetsDataInput(BaseModel):
    """Input schema for GoogleSheetsDataTool."""
    spreadsheet_name: str = Field(..., description="The name of the Google Sheet (e.g., 'automobile_data').")
    analysis_query: str = Field(..., description="Natural language query for analyzing the dataframe (e.g., 'calculate total sales', 'show summary statistics', 'group by make').")

class GoogleSheetsDataTool(BaseTool):
    name: str = "google_sheets_data_tool"
    description: str = "Useful for getting data from a Google Sheet and performing analysis on it. Requires the spreadsheet name and a natural language query for analysis."
    args_schema: Type[GoogleSheetsDataInput] = GoogleSheetsDataInput

    def _run(self, spreadsheet_name: str, analysis_query: str):
        if client is None:
            return "Error: Google Sheets credentials not loaded. Cannot access spreadsheet."

        try:
            spreadsheet = client.open(spreadsheet_name)
            worksheet = spreadsheet.sheet1  # Get the first sheet
            data = worksheet.get_all_records()
            df = pd.DataFrame(data)

            # Convert numeric columns to appropriate types
            numeric_columns = ['price', 'sales', 'mileage', 'engine_size', 'horsepower', 'city_mpg', 'highway_mpg']
            for col in numeric_columns:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')

            # Enhanced analysis logic based on the analysis_query
            query_lower = analysis_query.lower()

            if "total price" in query_lower or "ยอดขายรวม" in query_lower:
                if 'price' in df.columns:
                    total_price = df['price'].sum()
                    return f"Total price: {total_price:,.2f}"
                else:
                    return "Error: 'price' column not found in the dataframe."

            elif "average price" in query_lower or "ยอดขายเฉลี่ย" in query_lower:
                if 'price' in df.columns:
                    average_price = df['price'].mean()
                    return f"Average price: {average_price:,.2f}"
                else:
                    return "Error: 'price' column not found in the dataframe."

            elif "summary" in query_lower or "describe" in query_lower:
                return df.describe().to_string()

            elif "shape" in query_lower or "size" in query_lower:
                return f"DataFrame shape: {df.shape[0]} rows × {df.shape[1]} columns"

            elif "columns" in query_lower or "column names" in query_lower:
                return f"Column names: {list(df.columns)}"

            elif "group by make" in query_lower:
                if 'make' in df.columns:
                    if 'price' in df.columns:
                        grouped = df.groupby('make')['price'].agg(['count', 'mean', 'sum']).round(2)
                        return grouped.to_string()
                    else:
                        grouped = df.groupby('make').size()
                        return f"Count by make:\n{grouped.to_string()}"
                else:
                    return "Error: 'make' column not found in the dataframe."

            elif "fuel type" in query_lower or "group by fuel" in query_lower:
                if 'fuel_type' in df.columns:
                    grouped = df.groupby('fuel_type').size()
                    return f"Count by fuel type:\n{grouped.to_string()}"
                else:
                    return "Error: 'fuel_type' column not found in the dataframe."

            elif "head" in query_lower or "first" in query_lower:
                return df.head().to_string()

            elif "tail" in query_lower or "last" in query_lower:
                return df.tail().to_string()

            elif "missing" in query_lower or "null" in query_lower:
                missing_data = df.isnull().sum()
                return f"Missing values per column:\n{missing_data.to_string()}"

            elif "get dataframe" in query_lower or "get data" in query_lower:
                 return df.to_string()

            else:
                # Default response with basic info and suggestions
                return f"""Analysis query '{analysis_query}' not specifically recognized.

DataFrame info:
- Shape: {df.shape[0]} rows × {df.shape[1]} columns
- Columns: {list(df.columns)}

Available analysis options:
- 'total price' or 'average price'
- 'summary' or 'describe' for statistics
- 'group by make' or 'group by fuel type'
- 'head' or 'tail' to see first/last rows
- 'missing' to check for null values
- 'columns' to see column names
- 'get dataframe' or 'get data' to see the full data"""

        except Exception as e:
            return f"Error accessing Google Sheet or performing analysis: {e}"

# Qwen/Qwen3-235B-A22B-Thinking-2507 - นานมาก 0.20
# Qwen/Qwen2.5-72B-Instruct-Turbo - เร็ว

llm = LLM(model="together_ai/Qwen/Qwen2.5-72B-Instruct-Turbo",
          api_key="tgp_v1_X6sy0ZuipjZK1zAmpXpIM6XwPlE7pks6xiZBftKPu9Y",
          base_url="https://api.together.xyz/v1"
        )

test_agent = Agent(
    llm = llm,
    role="Thai Research Analyst",
    goal="Find and summarize information about specific topics, including analyzing price data from Google Sheets.",
    backstory="You are an experienced researcher with attention to detail and strong analytical skills, especially with sales data.",
    verbose=True,
    tools=[TimeTool(), GoogleSheetsDataTool()] # Update tools to use the new merged tool
)

## ลองเขียนให้เป็นฟังก์ชัน ที่เรียกใช้ง่ายๆ

In [72]:
def ask_llm_with_tool(question: str):
    """
    Takes a question as input, creates a task for the research_agent with the question,
    and runs the crew to get the answer.

    Args:
        question (str): The question to ask the LLM.

    Returns:
        str: The output from the crew's execution.
    """
    # Create a new task with the user's question
    question_task = Task(
        description=question,
        expected_output="An answer to the question based on available tools and agent's capabilities.",
        agent=test_agent # Use the already defined test_agent
    )

    # Create a new crew with the existing agent and the new task
    question_crew = Crew(
        agents=[test_agent],
        tasks=[question_task],
        verbose=True
    )

    # Run the crew and return the result
    result = question_crew.kickoff()
    return result

# Example usage (you can call this function with your questions)
# answer = ask_llm_with_tool("What is the current time in Thailand?")
# print(answer)

## Multi-Agent ASR Text Processing Function

This function receives text from ASR (Automatic Speech Recognition) and processes it through the CrewAI multi-agent system.

In [73]:
answer = ask_llm_with_tool("Give me the dataframe of Google Sheets")
print(answer)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3e81a080-7a0d-4c05-b15a-1215e3fb3fad                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Thai Research Analyst                                                                                   │
│                                                                                                                 │
│  Task: Give me the dataframe of Google Sheets                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Thai Research Analyst                                                                                   │
│                                                                                                                 │
│  Thought: I need to access the Google Sheets data to provide the complete content of the dataframe as           │
│  requested.                                                                                                     │
│                                                                                                                 │
│  Using Tool: google_sheets_data_tool                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"spreadsheet_name\": \"automobile_data\", \"analysis_query\": \"show all data\"}"                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Thai Research Analyst                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The complete dataframe from the Google Sheets 'automobile_data' is as follows:                                 │
│                                                                                                                 │
│  ```                                                                                                            │
│                 make fuel_type num_of_doors   body_style drive_wheels engine_location  wheel_base  length       │
│  width  height  curb_weight engine_type num_of_cylinders  engine_size fuel_system  compression_ratio            │
│  horsepower  city_mpg  highway_mpg  price                                                                       │
│  0      alfa-romero       gas          two  convertible          rwd           front        88.6   168.8        │
│  64.1    48.8         2548        dohc             four          130        mpfi               9.00             │
│  111        21           27  13495                                                                              │
│  1      alfa-romero       gas          two  convertible          rwd           front        88.6   168.8        │
│  64.1    48.8         2548        dohc             four          130        mpfi               9.00             │
│  111        21           27  16500                                                                              │
│  2      alfa-romero       gas          two    hatchback          rwd           front        94.5   171.2        │
│  65.5    52.4         2823        ohcv              six          152        mpfi               9.00             │
│  154        19           26  16500                                                                              │
│  3             audi       gas         four        sedan          fwd           front        99.8   176.6        │
│  66.2    54.3         2337         ohc             four          109        mpfi              70.00             │
│  102        24           30  13950                                                                              │
│  4             audi       gas         four        sedan          4wd           front        99.4   176.6        │
│  66.4    54.3         2824         ohc             five          136        mpfi               8.00             │
│  115        18           22  17450                                                                              │
│  5             audi       gas          two        sedan          fwd           front        99.8   177.3        │
│  66.3    53.1         2507         ohc             five          136        mpfi               8.50             │
│  110        19           25  15250                                                                              │
│  6             audi       gas         four        sedan          fwd           front       105.8   192.7        │
│  71.4    55.7         2844         ohc             five          136        mpfi               8.50             │
│  110        19           25  17710                                                                              │
│  7             audi       gas         four        wagon          fwd           front       105.8   192.7        │
│  71.4    55.7         2954         ohc             five          136        mpfi               8.50             │
│  110        19           25  18920                     

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d909c3e0-d844-47f3-8b88-fbe85d6b8b73                                                                     │
│  Agent: Thai Research Analyst                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3e81a080-7a0d-4c05-b15a-1215e3fb3fad                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The complete dataframe from the Google Sheets 'automobile_data' is as follows:                   │
│                                                                                                                 │
│  ```                                                                                                            │
│                 make fuel_type num_of_doors   body_style drive_wheels engine_location  wheel_base  length       │
│  width  height  curb_weight engine_type num_of_cylinders  engine_size fuel_system  compression_ratio            │
│  horsepower  city_mpg  highway_mpg  price                                                                       │
│  0      alfa-romero       gas          two  convertible          rwd           front        88.6   168.8        │
│  64.1    48.8         2548        dohc             four          130        mpfi               9.00             │
│  111        21           27  13495                                                                              │
│  1      alfa-romero       gas          two  convertible          rwd           front        88.6   168.8        │
│  64.1    48.8         2548        dohc             four          130        mpfi               9.00             │
│  111        21           27  16500                                                                              │
│  2      alfa-romero       gas          two    hatchback          rwd           front        94.5   171.2        │
│  65.5    52.4         2823        ohcv              six          152        mpfi               9.00             │
│  154        19           26  16500                                                                              │
│  3             audi       gas         four        sedan          fwd           front        99.8   176.6        │
│  66.2    54.3         2337         ohc             four          109        mpfi              70.00             │
│  102        24           30  13950                                                                              │
│  4             audi       gas         four        sedan          4wd           front        99.4   176.6        │
│  66.4    54.3         2824         ohc             five          136        mpfi               8.00             │
│  115        18           22  17450                                                                              │
│  5             audi       gas          two        sedan          fwd           front        99.8   177.3        │
│  66.3    53.1         2507         ohc             five          136        mpfi               8.50             │
│  110        19           25  15250                                                                              │
│  6             audi       gas         four        sedan          fwd           front       105.8   192.7        │
│  71.4    55.7         2844         ohc             five          136        mpfi               8.50             │
│  110        19           25  17710                                                                              │
│  7             audi       gas         four        wagon          fwd           front       105.8   192.7        │
│  71.4    55.7         2954         ohc             fiv

The complete dataframe from the Google Sheets 'automobile_data' is as follows:

```
               make fuel_type num_of_doors   body_style drive_wheels engine_location  wheel_base  length  width  height  curb_weight engine_type num_of_cylinders  engine_size fuel_system  compression_ratio  horsepower  city_mpg  highway_mpg  price
0      alfa-romero       gas          two  convertible          rwd           front        88.6   168.8   64.1    48.8         2548        dohc             four          130        mpfi               9.00         111        21           27  13495
1      alfa-romero       gas          two  convertible          rwd           front        88.6   168.8   64.1    48.8         2548        dohc             four          130        mpfi               9.00         111        21           27  16500
2      alfa-romero       gas          two    hatchback          rwd           front        94.5   171.2   65.5    52.4         2823        ohcv              six          152 

In [74]:
answer = ask_llm_with_tool("ขอยอดขายรวมทั้งหมดจากข้อมูล automobile_data")
print(answer)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0ad08176-4ff4-422a-833d-0c5bfad54491                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Thai Research Analyst                                                                                   │
│                                                                                                                 │
│  Task: ขอยอดขายรวมทั้งหมดจากข้อมูล automobile_data                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Thai Research Analyst                                                                                   │
│                                                                                                                 │
│  Thought: Thought: I need to use the google_sheets_data_tool to access the 'automobile_data' spreadsheet and    │
│  calculate the total sales.                                                                                     │
│                                                                                                                 │
│  Using Tool: google_sheets_data_tool                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"spreadsheet_name\": \"automobile_data\", \"analysis_query\": \"calculate total sales\"}"                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Thai Research Analyst                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ยอดขายรวมทั้งหมดจากข้อมูล automobile_data คือ 2,635,443.00                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9116d876-c688-4276-8bc6-551fd4eda512                                                                     │
│  Agent: Thai Research Analyst                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0ad08176-4ff4-422a-833d-0c5bfad54491                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ยอดขายรวมทั้งหมดจากข้อมูล automobile_data คือ 2,635,443.00                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ยอดขายรวมทั้งหมดจากข้อมูล automobile_data คือ 2,635,443.00


In [75]:
answer = ask_llm_with_tool("ขอสรุปข้อมูลของ dataframe หน่อย")
print(answer)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 961ec6f9-bad2-406a-90d2-2ad73480374e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Thai Research Analyst                                                                                   │
│                                                                                                                 │
│  Task: ขอสรุปข้อมูลของ dataframe หน่อย                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Thai Research Analyst                                                                                   │
│                                                                                                                 │
│  Thought: I need to use the google_sheets_data_tool to get the data and perform the analysis as requested.      │
│                                                                                                                 │
│  Using Tool: google_sheets_data_tool                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"spreadsheet_name\": \"automobile_data\", \"analysis_query\": \"show summary statistics\"}"                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Thai Research Analyst                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ข้อมูลสรุปของ dataframe ดังนี้:                                                                                      │
│                                                                                                                 │
│  ```                                                                                                            │
│          wheel_base      length       width      height  curb_weight  engine_size  compression_ratio            │
│  horsepower    city_mpg  highway_mpg         price                                                              │
│  count  203.000000  203.000000  203.000000  203.000000   203.000000   203.000000         203.000000             │
│  203.000000  203.000000   203.000000    203.000000                                                              │
│  mean    98.782759  173.999015   65.901478   53.733498  2555.921182   126.857143          10.452315             │
│  104.256158   25.241379    30.748768  12982.477833                                                              │
│  std      6.045680   12.385511    2.154835    2.442864   523.205555    41.845239           5.792528             │
│  39.714369    6.570702     6.920406   8111.953571                                                               │
│  min     86.600000  141.100000   60.300000   47.800000  1488.000000    61.000000           7.000000             │
│  48.000000   13.000000    16.000000      0.000000                                                               │
│  25%     94.500000  166.300000   64.050000   52.000000  2145.000000    97.000000           8.550000             │
│  70.000000   19.000000    25.000000   7649.000000                                                               │
│  50%     97.000000  173.200000   65.500000   54.100000  2414.000000   119.000000           9.000000             │
│  95.000000   24.000000    30.000000  10245.000000                                                               │
│  75%    102.400000  183.300000   66.900000   55.500000  2943.500000   143.000000           9.400000             │
│  116.000000   30.000000    35.000000  16500.000000                                                              │
│  max    120.900000  208.100000   72.300000   59.800000  4066.000000   326.000000          70.000000             │
│  288.000000   49.000000    54.000000  45400.000000                                                              │
│  ```                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ec6bd101-a5d6-4c72-88af-c5702f9a25cf                                                                     │
│  Agent: Thai Research Analyst                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 961ec6f9-bad2-406a-90d2-2ad73480374e                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ข้อมูลสรุปของ dataframe ดังนี้:                                                                        │
│                                                                                                                 │
│  ```                                                                                                            │
│          wheel_base      length       width      height  curb_weight  engine_size  compression_ratio            │
│  horsepower    city_mpg  highway_mpg         price                                                              │
│  count  203.000000  203.000000  203.000000  203.000000   203.000000   203.000000         203.000000             │
│  203.000000  203.000000   203.000000    203.000000                                                              │
│  mean    98.782759  173.999015   65.901478   53.733498  2555.921182   126.857143          10.452315             │
│  104.256158   25.241379    30.748768  12982.477833                                                              │
│  std      6.045680   12.385511    2.154835    2.442864   523.205555    41.845239           5.792528             │
│  39.714369    6.570702     6.920406   8111.953571                                                               │
│  min     86.600000  141.100000   60.300000   47.800000  1488.000000    61.000000           7.000000             │
│  48.000000   13.000000    16.000000      0.000000                                                               │
│  25%     94.500000  166.300000   64.050000   52.000000  2145.000000    97.000000           8.550000             │
│  70.000000   19.000000    25.000000   7649.000000                                                               │
│  50%     97.000000  173.200000   65.500000   54.100000  2414.000000   119.000000           9.000000             │
│  95.000000   24.000000    30.000000  10245.000000                                                               │
│  75%    102.400000  183.300000   66.900000   55.500000  2943.500000   143.000000           9.400000             │
│  116.000000   30.000000    35.000000  16500.000000                                                              │
│  max    120.900000  208.100000   72.300000   59.800000  4066.000000   326.000000          70.000000             │
│  288.000000   49.000000    54.000000  45400.000000                                                              │
│  ```                                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ข้อมูลสรุปของ dataframe ดังนี้:

```
        wheel_base      length       width      height  curb_weight  engine_size  compression_ratio  horsepower    city_mpg  highway_mpg         price
count  203.000000  203.000000  203.000000  203.000000   203.000000   203.000000         203.000000  203.000000  203.000000   203.000000    203.000000
mean    98.782759  173.999015   65.901478   53.733498  2555.921182   126.857143          10.452315  104.256158   25.241379    30.748768  12982.477833
std      6.045680   12.385511    2.154835    2.442864   523.205555    41.845239           5.792528   39.714369    6.570702     6.920406   8111.953571
min     86.600000  141.100000   60.300000   47.800000  1488.000000    61.000000           7.000000   48.000000   13.000000    16.000000      0.000000
25%     94.500000  166.300000   64.050000   52.000000  2145.000000    97.000000           8.550000   70.000000   19.000000    25.000000   7649.000000
50%     97.000000  173.200000   65.500000   54.100000  2414.00

## Note : สร้าง Tools สำหรับ Google Sheets

* วิธี สร้าง client_secret.json ใช้วิธีตามนี้ : https://calvinowens.medium.com/how-to-manage-google-sheets-with-python-and-gspread-7158cb6a444b

## MCP ... To be continued

https://docs.crewai.com/en/mcp/overview